In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from keras.applications import ResNet50
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.utils import load_img, img_to_array
import requests








In [4]:
# CONSTANTS
FOLDERS = {'train': '/content/drive/My Drive/big_brother_firefighter/train', 'test':'/content/drive/My Drive/big_brother_firefighter/test'}
PIC_SIZE = 224
NUM_EPOCHS = 20
NUM_CLASSES = 3
TRAIN_BATCH_SIZE = 100
TEST_BATCH_SIZE =1 

In [ ]:
def init_model():
  """
  Initializing a sequential model, 1st ResNet50, 2nd dense 100, 3rd dense 3
  """
  mymodel = Sequential()
  resnet = ResNet50(include_top=False, pooling='avg')
  mymodel.add(resnet)
  mymodel.layers[0].trainable = False # No need to train Resnet50 
  mymodel.add(Dense(100, activation='relu'))
  mymodel.add(Dense(NUM_CLASSES, activation='softmax')) # 3 is the amount of labels
  mymodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return mymodel

In [ ]:
# Connect to Google Colab folder for training.
from google.colab import drive
drive.mount('/content/drive')


In [6]:
# Training Model Part
def training(model):
  """
  Returns a trained model, the history of the model, and a labels dictionary.
  Training the model with images from google drive.
  """
  data_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                zoom_range=[0.9, 1.25],
                                horizontal_flip=True,
                                vertical_flip=False,
                                data_format='channels_last',
                                brightness_range=[0.5, 1.5]
                               )
  train_gen = data_gen.flow_from_directory(FOLDERS['train'], 
                                            target_size= (PIC_SIZE,PIC_SIZE),
                                            batch_size=TRAIN_BATCH_SIZE,
                                            shuffle = False,
                                            class_mode='categorical')
  labels = train_gen.class_indices 
  labels_dict = {v:k for k,v in labels.items()}
  val_gen = data_gen.flow_from_directory(FOLDERS['test'], 
                                            target_size= (PIC_SIZE,PIC_SIZE),
                                            batch_size=TEST_BATCH_SIZE,
                                            shuffle = False,
                                            class_mode='categorical')
  n_train = train_gen.n
  
  model_history = model.fit(
            train_gen,
            steps_per_epoch=n_train/TRAIN_BATCH_SIZE,
            epochs=NUM_EPOCHS,
            validation_data=val_gen,
            validation_steps=1                 
            )
  return model, model_history, labels_dict


In [ ]:
# Initilaize basic model before training
model = init_model()

model.summary()

In [5]:
# Returns model as mymodel, model history, and the labels from the dictionary
mymodel, model_history, labels_dict = training(model)

NameError: ignored

In [ ]:
# Plotting the model accuracy and loss for refrence.
def plot_history( model_history ):
  """
  Plots the accuracy, and loss of the model on the train and val 
  using the model history.
  """
    plt.style.use("ggplot")
    fig = plt.figure()
    fig.set_size_inches(15, 5)
    
    fig.add_subplot(1, 3, 1)
    plt.plot(np.arange(0, NUM_EPOCHS), model_history.history["loss"], label="train_loss")
    plt.plot(np.arange(0, NUM_EPOCHS), model_history.history["val_loss"], label="val_loss")
    plt.title("Training Loss and Validation Loss on Dataset")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss")
    plt.legend(loc="lower left")

    
    fig.add_subplot(1, 3, 2)
    plt.plot(np.arange(0, NUM_EPOCHS), model_history.history["loss"], label="train_loss")
    plt.plot(np.arange(0, NUM_EPOCHS), model_history.history["accuracy"], label="train_acc")
    plt.title("Training Loss and Accuracy on Dataset")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    
    fig.add_subplot(1, 3, 3)
    plt.plot(np.arange(0, NUM_EPOCHS), model_history.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, NUM_EPOCHS), model_history.history["val_accuracy"], label="val_aaccuracy")
    plt.title("Validation Loss and Accuracy on Dataset")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")


    plt.show()
    #plt.savefig("plot.png")
  plot_history(model_history)

In [ ]:
# Predict the images using url link
import requests
from PIL import Image
from io import BytesIO
def predict_from_url(model, url, label_dict):
  """
  Returns the label with the highest confidence.
  Returns a tuple with all the confidence levels.
  """
  response = requests.get(url)
  image = Image.open(BytesIO(response.content))
  image = image.resize((PIC_SIZE, PIC_SIZE))
  image_array = img_to_array(image)
  image_array = np.expand_dims(image_array, axis=0)
  image_array = preprocess_input(image_array)
  pred_vec = model.predict(image_array)
  pred_class = np.argmax(pred_vec)
  confidence = np.max(pred_vec)
  label = label_dict[pred_class]
  return label, confidence, pred_vec


In [7]:
# Change img_path to get the prediction.
img_path = 'https://img.freepik.com/premium-psd/snowdrift-isolated-transparent-background_725966-38.jpg?w=2000'
predict_from_url(mymodel, img_path, labels_dict)

NameError: ignored